In [ ]:
#5.4.1. 不带参数的层
#首先，我们构造一个没有任何参数的自定义层。如果你还记得我们在 5.1节 对块的介绍，
#这应该看起来很眼熟。下面的CenteredLayer类要从其输入中减去均值。
#要构建它，我们只需继承基础层类并实现正向传播功能。
import torch
import torch.nn.functional as F
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

In [ ]:
#让我们通过向其提供一些数据来验证该层是否按预期工作。
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [ ]:
#我们可以将层作为组件合并到构建更复杂的模型中。
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

In [ ]:
#作为额外的健全性检查，我们可以向网络发送随机数据后，检查均值是否为0。由于我们处理的是浮点数，
#因为存储精度的原因，我们仍然可能会看到一个非常小的非零数。
Y = net(torch.rand(4, 8))
Y.mean()

tensor(5.5879e-09, grad_fn=<MeanBackward0>)

In [ ]:
#5.4.2. 带参数的图层
#现在，让我们实现自定义版本的全连接层。回想一下，该层需要两个参数，一个用于表示权重，
#另一个用于表示偏置项。在此实现中，我们使用ReLU作为激活函数。该层需要输入参数：in_units和units，分别表示输入和输出的数量。
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))

    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [ ]:
#实例化MyDense类并访问其模型参数。
dense = MyLinear(5, 3)
dense.weight

Parameter containing:
tensor([[-0.4620, -0.1337, -0.0640],
        [-0.8994,  2.4415,  0.2044],
        [ 0.0836,  0.4356,  0.7880],
        [-1.0027, -0.8975, -1.6845],
        [-0.2940,  0.5066, -0.3934]], requires_grad=True)

In [ ]:
#可以使用自定义层直接执行正向传播计算。
dense(torch.rand(2, 5))

tensor([[0.0000, 0.1683, 0.0000],
        [0.0000, 0.2254, 0.0000]])

In [ ]:
#可以使用自定义层构建模型。我们可以像使用内置的全连接层一样使用自定义层。
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[5.7780],
        [0.7684]])